# Predicciones (en desarrollo)
> Modelos predictivos

- toc: false 
- badges: false
- comments: false

## Real (puntos) y predicción (curva)

In [3]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt
import datetime

# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

df = df[30:34][['Fecha','Total confirmados']]
df['dias'] = np.array(range(len(df)))
df['log(Total confirmados)'] = np.log(df['Total confirmados'])
df = df.reset_index(drop=True)

y = np.array(df['log(Total confirmados)'])
x = np.array(df['dias'])
a,b = np.polyfit(x, y, 1, w=np.sqrt(y))

preds = []
pred_span = 7
for i in range(len(x)+pred_span):
    log_pred = a*i + b
    pred = np.e**log_pred
    preds.append(pred)
    
df['modelo'] = preds[0:len(x)]

for ii in range(pred_span):
    i = ii+len(x)
    df.loc[i,'Fecha'] = df.loc[i-1,'Fecha'] + datetime.timedelta(days=1)
    df.loc[i,'dias'] = df.loc[i-1,'dias']
    df.loc[i,'modelo'] = preds[ii+len(x)]
    
source = df
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['modelo'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='monthdate(Fecha):O',
    y='modelo:Q',
    color=alt.value("#FF0000")
)

scatter = alt.Chart(source).mark_circle(size=60).encode(
    x='monthdate(Fecha):O',
    y='Total confirmados',
    tooltip=['Total confirmados']
).interactive()

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='monthdate(Fecha):O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'modelo:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='monthdate(Fecha):O',
).transform_filter(
    nearest
)


# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text, scatter
).properties(
    width=800, height=400
).configure(
    padding=20
)

alt.LayerChart(...)

In [4]:
# collapse
df[['Fecha', 'Total confirmados', 'modelo']]

,Fecha,Total confirmados,modelo
0,2020-02-29,62.0,61.290368
1,2020-03-01,83.0,85.448516
2,2020-03-02,122.0,119.128815
3,2020-03-03,165.0,166.084505
4,2020-03-04,NaN,231.548202
5,2020-03-05,NaN,322.815002
6,2020-03-06,NaN,450.055429
7,2020-03-07,NaN,627.448811
8,2020-03-08,NaN,874.763386
9,2020-03-09,NaN,1219.559218
